論文 
https://arxiv.org/abs/2003.03808v3  
GitHub  
https://github.com/adamian98/pulse  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/PULSE_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# GitHubからPULSEのソースコードを取得

In [ ]:
%cd /content/
!git clone https://github.com/adamian98/pulse.git

# ライブラリのインポート

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import glob
import os

# テスト画像のセットアップ

In [ ]:
%cd /content/pulse
!rm -rf ./origin
!mkdir ./origin
%cd /content/pulse/origin

image_type ='sample' #@param ['sample', 'upload']
if image_type == 'sample':
  !wget https://www.pakutaso.com/shared/img/thumb/YK0I9A6188_TP_V4.jpg
  file_name = "YK0I9A6188_TP_V4.jpg"
else:
  uploaded = files.upload()
  uploaded = list(uploaded.keys())
  file_name = uploaded[0]

image = Image.open(file_name).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# 低解像度顔画像生成

In [ ]:
%cd /content/pulse/
# 顔部分を切り出した低解像度画像出力先
align_dir = "./input_face"

!rm -rf {align_dir}

!python align_face.py \
  -input_dir ./origin \
  -output_dir {align_dir} \
  -output_size 32 \
  -seed 1234 \

align_imgs = glob.glob(align_dir + "/*.png") 
image = Image.open(align_imgs[0]).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# 超解像

In [ ]:
%cd /content/pulse/
sr_result = "./sr_result"
steps = 100

!rm -rf {sr_result}

!python run.py \
  -input_dir {align_dir} \
  -output_dir {sr_result} \
  -duplicates 1 \
  -seed 1234 \
  -noise_type trainable \
  -steps {steps} \
  -save_intermediate

In [ ]:
file_name_wo_ext = os.path.splitext(os.path.basename(file_name))[0]
hr_file_name = sr_result + "/" + file_name_wo_ext + "_0/HR/" + file_name_wo_ext + "_0_" + str(steps-1) + ".png"

image = Image.open(hr_file_name).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# テスト動画のセットアップ

In [ ]:
%cd /content/pulse
!rm -rf ./origin_video
!mkdir -p ./origin_video/frames
%cd /content/pulse/origin_video

image_type ='sample' #@param ['sample', 'upload']
if image_type == 'sample':
  !wget https://vod-progressive.akamaized.net/exp=1644062740~acl=%2Fvimeo-prod-skyfire-std-us%2F01%2F4172%2F18%2F470864736%2F2095514120.mp4~hmac=25a8ca2e8e3028808e32b3a9fa323820cd8c3ca5e3000e8314a8d8454923b4e4/vimeo-prod-skyfire-std-us/01/4172/18/470864736/2095514120.mp4?download=1&filename=pexels-diva-plavalaguna-5664462.mp4
  !mv 2095514120.mp4?download=1 2095514120.mp4
  video_name = "2095514120.mp4"
else:
  uploaded = files.upload()
  uploaded = list(uploaded.keys())
  video_name = uploaded[0]

from moviepy.editor import *
from moviepy.video.fx.resize import resize

clip = VideoFileClip(video_name)
# 2~3秒までを切り出し
sub_clip = clip.subclip(2, 3)
sub_clip.write_videofile("src-video.mp4")
video_name = "src-video.mp4"

# 表示用にリサイズ
clip_resize = resize(sub_clip, height=120)
clip_resize.ipython_display()

# テスト動画をフレーム分割

In [ ]:
!ffmpeg -i {video_name} /content/pulse/origin_video/frames/src-video-frame-%d.png

# 低解像度顔画像生成

In [ ]:
%cd /content/pulse/
# 顔部分を切り出した低解像度画像出力先
video_align_dir = "./video_input_face"

!rm -rf {video_align_dir}

!python align_face.py \
  -input_dir ./origin_video/frames \
  -output_dir {video_align_dir} \
  -output_size 32 \
  -seed 1234 \

video_align_imgs = glob.glob(video_align_dir + "/*.png") 
image = Image.open(video_align_imgs[0]).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# 超解像(動画)

In [ ]:
%cd /content/pulse/
sr_video_result = "./sr_video_result"
steps = 20

!rm -rf {sr_video_result}

!python run.py \
  -input_dir {video_align_dir} \
  -output_dir {sr_video_result} \
  -duplicates 1 \
  -seed 1234 \
  -noise_type trainable \
  -steps {steps} \
  -save_intermediate

# frame画像を収集
各フレーム画像の超解像結果を一つのディレクトリ配下にコピー

In [ ]:
%cd /content/pulse/
sr_video_result_frames = sr_video_result + "/frames"
!mkdir -p {sr_video_result_frames}

results = glob.glob(sr_video_result + "/*/HR/*_0_" + str(steps-1) + ".png")
print(results)

for result in results:
  !cp {result} {sr_video_result_frames}


# frame画像を動画に変換

In [ ]:
src_frames = sr_video_result_frames + "/src-video-frame-%0d_0_" + str(steps-1) + ".png"
dst_video = sr_video_result + "/out.mp4"

!ffmpeg -i {src_frames} -c:v libx264 -vf "fps=25,format=yuv420p" {dst_video}

clip = VideoFileClip(dst_video)
clip.ipython_display()

In [ ]:
orging_frames = video_align_dir + "/src-video-frame-%0d_0.png"
dst_orgin_video = sr_video_result + "/origin_out.mp4"

!ffmpeg -i {orging_frames} -c:v libx264 -vf "fps=25,format=yuv420p" {dst_orgin_video}

clip = VideoFileClip(dst_orgin_video)
clip_resize = resize(clip, height=128)
clip_resize.ipython_display()